In [1]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
import cv2
import numpy as np

class FeatureTracking:
    def __init__(self, video_path, output_path, output_fps):
        self.cap = cv2.VideoCapture(video_path)
        self.max_corners = 2000
        self.quality_level = 0.001
        self.min_distance = 0.1

        self.lk_params = dict(winSize=(15, 15), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

        self.frame_width = 1280
        self.frame_height = 720

        # Region of Interest
        self.square_size = (670, 720)
        self.square_x = int(self.frame_width - self.square_size[0]) // 2  # since the square will be placed relative to the top left corner of the video
        self.square_y = int(self.frame_height - self.square_size[1]) // 2  # we want to make sure that the ROI is placed in the middle
        self.square_position = (self.square_x, self.square_y)

        self.output_path = output_path
        self.output_fps = output_fps

    def process_video(self):
        ret, prev_frame = self.cap.read()
        prev_frame = cv2.resize(prev_frame, (self.frame_width, self.frame_height))  # Resize the frame
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=self.max_corners, qualityLevel=self.quality_level, minDistance=self.min_distance)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(self.output_path, fourcc, self.output_fps, (self.frame_width, self.frame_height))

        while True:
            for _ in range(3):
                ret, frame = self.cap.read()
                if not ret:
                    break

            # Convert to grayscale
            frame = cv2.resize(frame, (self.frame_width, self.frame_height))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Create ROI
            mask = np.zeros(frame.shape[:2], dtype=np.uint8)
            x, y, w, h = self.square_position[0], self.square_position[1], self.square_size[0], self.square_size[1]
            mask[y:y + h, x:x + w] = 230
            masked_frame = cv2.bitwise_and(frame, frame, mask=mask)

            # Calculate optical flow using Lucas-Kanade on the masked_frame
            prev_frame = cv2.resize(prev_frame, (self.frame_width, self.frame_height))  # Resize the previous frame
            new_pts, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, **self.lk_params)

            valid_new_pts = new_pts[status == 1]
            valid_prev_pts = prev_pts[status == 1]

            for i, (new, prev) in enumerate(zip(valid_new_pts, valid_prev_pts)):
                a, b = new.ravel()
                c, d = prev.ravel()
                a, b, c, d = int(a), int(b), int(c), int(d)

                if self.square_position[0] <= a <= self.square_position[0] + self.square_size[0] and self.square_position[1] <= b <= self.square_position[1] + self.square_size[1]:
                    scaling_factor = 5
                    a_new = int(a - (a - c) * scaling_factor)
                    b_new = int(b - (b - d) * scaling_factor)

                    # Limit drawing to the ROI
                    mask = cv2.line(mask, (a, b), (a_new, b_new), (0, 255, 0), 2)
                    masked_frame = cv2.circle(masked_frame, (a, b), 3, (0, 255, 0), -1)

            result = cv2.add(frame, masked_frame)

            out.write(result)

            cv2.imshow('Frame', result)

            k = cv2.waitKey(30) & 0xff
            if k == 27:
                break

            prev_gray = gray.copy()
            prev_pts = valid_new_pts.reshape(-1, 1, 2)

        self.cap.release()
        out.release()
        cv2.destroyAllWindows()
            
if __name__ == "__main__":
    video_path = "G:\THESIS\SampleVideos\SMTracking_LongSleeve.mp4"
    output_path = "G:\THESIS\LKOF\OutputVideos\output_video.avi"
    output_fps = 1.7 * 30  # 1.5x speedup
    feature_tracker = FeatureTracking(video_path, output_path, output_fps)
    feature_tracker.process_video()


In [16]:

def track_features(video_path):
    
    cap = cv2.VideoCapture(video_path)

    max_corners = 3000
    quality_level = 0.001
    min_distance = 0.1 

    # Parameters for Lucas-Kanade Optical Flow
    lk_params = dict(winSize=(25, 25), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

    # Read the first frame
    ret, prev_frame = cap.read()
    prev_frame = cv2.resize(prev_frame, (1280, 720))  # Resize the frame
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=max_corners, qualityLevel=quality_level, minDistance=min_distance)

    frame_width = 1280
    frame_height = 720

    # ROI
    square_size = (670, 720) #ROI Size
    square_x = int(frame_width - square_size[0]) // 2
    square_y = int(frame_height - square_size[1]) // 2
    square_position = (square_x, square_y) 

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (1280, 720)) 

        gray = cv2.cvtColor (frame, cv2.COLOR_BGR2GRAY)

        # Create a mask that covers the square ROI
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        x, y, w, h = square_position[0], square_position[1], square_size[0], square_size[1]
        mask[y:y + h, x:x + w] = 230  

        
        masked_frame = cv2.bitwise_and(frame, frame, mask=mask)

        # Calculate optical flow using Lucas-Kanade on the masked_frame
        prev_frame = cv2.resize(prev_frame, (1280, 720))  # Resize the previous frame
        new_pts, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, **lk_params)

        valid_new_pts = new_pts[status == 1]
        valid_prev_pts = prev_pts[status == 1]

        for i, (new, prev) in enumerate(zip(valid_new_pts, valid_prev_pts)):
            a, b = new.ravel()
            c, d = prev.ravel()
            a, b, c, d = int(a), int(b), int(c), int(d)

            if square_position[0] <= a <= square_position[0] + square_size[0] and square_position[1] <= b <= square_position[1] + square_size[1]:
                scaling_factor = 5
                a_new = int(a - (a - c) * scaling_factor)
                b_new = int(b - (b - d) * scaling_factor)

                #limit drawing to the ROI
                mask = cv2.line(mask, (a, b), (a_new, b_new), (0, 255, 0), 2)
                masked_frame = cv2.circle(masked_frame, (a, b), 3, (0, 255, 0), -1)

        result = cv2.add(frame, masked_frame)

        cv2.imshow('Frame', result)

        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

        prev_gray = gray.copy()
        prev_pts = valid_new_pts.reshape(-1, 1, 2)

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "D:\MAPUA\Thesis\SampleVideos\SMTracking_LongSleeve.MOV"
    track_features(video_path)


Lucas-Kanade Optical Flow Tracking with Shi-Tomasi Corner Detection

In [3]:
import cv2
import numpy as np

class LucasKanade_OpticalFlow:
    def __init__(self, video_path, window_width=1280, window_height=720, 
                 max_corners=1000, quality_level=0.01, min_distance=0.1):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path)

        # Get the frame width and height from the video 
        self.frame_width = int(self.cap.get(3))
        self.frame_height = int(self.cap.get(4))

        # Parameters of Shi-Tomasi Corner Detection
        self.max_corners = max_corners
        self.quality_level = quality_level
        self.min_distance = min_distance
        self.feature_params = dict(maxCorners=max_corners, qualityLevel=quality_level, minDistance=min_distance)

        # Parameters of Lucas-Kanade Optical Flow
        self.lk_params = dict(winSize=(25, 25), maxLevel=3,
                              criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                                        30, 0.01))

        # Initialize Variables for LK
        self.prev_frame = None
        self.prev_gray = None
        self.prev_corners = None

        # Set window size and position
        self.window_width = window_width
        self.window_height = window_height
        self.position_window()

    def position_window(self):
        cv2.namedWindow('Feature Tracking', cv2.WINDOW_NORMAL)
        cv2.moveWindow('Feature Tracking', (1920 - self.window_width) // 2, (1080 - self.window_height) // 2)
        cv2.resizeWindow('Feature Tracking', self.window_width, self.window_height)

    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, mask=None, **self.feature_params)
        return corners, gray

    def track_features(self):
        # Read the first frame
        ret, self.prev_frame = self.cap.read()
        self.prev_corners, self.prev_gray = self.detect_corners(self.prev_frame)

        # Create a mask for drawing purposes
        mask = np.zeros_like(self.prev_frame)

        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            # Detect features in the current frame using Shi-Tomasi corner detection
            new_corners, gray = self.detect_corners(frame)

            # Calculate optical flow using Lucas-Kanade
            new_corners, status, err = cv2.calcOpticalFlowPyrLK(self.prev_gray, gray, self.prev_corners, None,
                                                               **self.lk_params)

            # Select good points
            good_new = new_corners[status == 1]
            good_old = self.prev_corners[status == 1]

            # Draw tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 255, 0), -1)

            img = cv2.add(frame, mask)

            cv2.imshow('Feature Tracking', img)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            # Update previous frame and points
            self.prev_gray = gray.copy()
            self.prev_corners = good_new.reshape(-1, 1, 2)

        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "D:\MAPUA\Thesis\SampleVideos\SMTracking_LongSleeve.MOV"
    feature_tracker = LucasKanade_OpticalFlow(video_path, window_width=1280, window_height=720)
    feature_tracker.track_features()
